# 📁 Preparación de Datos - Sign2Speech

<div style="color: red; font-weight: bold;">
IMPORTANTE: Este Notebook se ejecute en el entorno Kaggle, dentro del marco de la competición de ASL Google para tener acceso a la base de datos
</div>

En este notebook se lleva a cabo el proceso de preparación de los datos del proyecto **Sign2Speech**, cuyo objetivo es traducir lenguaje de signos a sonido mediante técnicas de visión por computador y edge computing.

Se trabaja con el dataset **ASL** (Word-Level American Sign Language hecha por Google) y se construye una base de datos estructurada que contenga las secuencias de landmarks necesarias para entrenar el modelo de reconocimiento. Esta base de datos se guarda en formato `.npy` y está diseñada para estar alojada en **Kaggle**, de modo que pueda ser accedida de forma remota desde cualquier entorno de entrenamiento o despliegue.

Se incluyen las siguientes etapas:

- Carga y exploración inicial del dataset ASL.
- preprocessamiento del dataset.
- División del dataset en subconjuntos de entrenamiento y validación.
- Guardado en disco y subida a Kaggle para acceso remoto.

Este paso es fundamental para poder acceder a nuestros datos desde multiples entornos de entrenamiento.

In [ ]:
# Librerías estándar
import json

# Manejo de datos
import numpy as np
import pandas as pd

# Machine Learning
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split, GroupShuffleSplit 

# Visualización
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sn

# Utilidades
from tqdm.notebook import tqdm
import scipy


## ⚙️ Variables Globales

Se definen los parámetros principales que controlan el comportamiento del preprocesamiento, como el tamaño de entrada, número de dimensiones por landmark, control de aleatoriedad y flags de ejecución.

In [ ]:
# Si es True, se preprocesarán todos los datos desde cero.
PREPROCESS_DATA = True

# Número total de landmarks por frame (se ajusta según el subconjunto utilizado)
N_ROWS = 543

# Dimensiones por landmark: x, y, z
DIMS = 3
COLUMNS = ['x', 'y', 'z']

# Longitud objetivo para cada secuencia (en frames)
INPUT_SIZE = 64

# Máximo gap de frames consecutivos vacíos que se puede interpolar
GAP = 8

# Semilla para reproducibilidad
SEED = 42

In [ ]:
# Si no vamos a preprocesar de nuevo (modo rápido), tomamos una muestra de 5000 ejemplos para acelerar la ejecución.
# Si PREPROCESS_DATA es True, se carga todo el dataset completo.
if not PREPROCESS_DATA:
    train = pd.read_csv('/kaggle/input/asl-signs/train.csv').sample(int(5e3), random_state=SEED)
else:
    train = pd.read_csv('/kaggle/input/asl-signs/train.csv')

# Guardamos el número total de muestras cargadas
N_SAMPLES = len(train)
print(f'N_SAMPLES: {N_SAMPLES}')

## 📊 Distribución de Frecuencia de Signos en el Dataset

In [ ]:

# Contar cuántas veces aparece cada signo
sign_counts = train['sign'].value_counts().reset_index()
sign_counts.columns = ['sign', 'count']

# Graficar
plt.figure(figsize=(12, 6))
plt.bar(range(len(sign_counts)), sign_counts['count'])
plt.ylabel('Número de instancias')
plt.title('Instance Count per Gloss (Google ASL)')
plt.xticks([], [])  # Ocultar etiquetas en el eje x
plt.tight_layout()


# Guardar la imagen
plt.savefig('/kaggle/working/distribucion_signos.png')  # Se guarda en el directorio de trabajo
plt.show()

In [ ]:
# Estadísticas Descriptivas de la Frecuencia de Signos
sign_counts.describe()

## Preparamos directorios de nuestros ficheros Parquet

In [ ]:
# Añadimos la direction de cada fichero a la tabla train

def get_file_path(path):
    return f'/kaggle/input/asl-signs/{path}'

train['file_path'] = train['path'].apply(get_file_path)

In [ ]:
# Encodamos los signos
train['sign_ord'] = train['sign'].astype('category').cat.codes

# Dicc de enco 
SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()

In [ ]:
# Registramos el encodamiento de nuestros signos

with open("ord2sign.json", "w") as f:
    json.dump(ORD2SIGN, f)


In [ ]:
display(train.head(10))
display(train.info())

## Ejemplo:

In [ ]:
# Ejemplo de fichero parquet

ejemplo = train['file_path'][0]
display(ejemplo)
df  = pd.read_parquet(ejemplo)
display(df.head(10))

# Procesamiento de los datos:

## 🧍‍♂️ Landmarks Seleccionados

La variable `LANDMARK_IDX` define un conjunto reducido de índices de landmarks que se utilizarán como entrada al modelo. Estos puntos han sido seleccionados manualmente por su relevancia en la comunicación gestual y facial.

- **[0, 9, 11, 13, 14, 17]**: Puntos clave del cuerpo relacionados con hombros, codos y cuello.
- **[117, 118, 119, 199, 346, 347, 348]**: Landmarks de la cara, especialmente en la zona de los labios y ojos.
- **range(468, 543)**: Landmarks de las manos (MediaPipe define 21 puntos por mano en esta zona).

Este enfoque reduce la dimensionalidad del input y mejora la eficiencia sin sacrificar información crítica para el reconocimiento de signos.

In [ ]:
# landmarks utilizados
LANDMARK_IDX = [0,9,11,13,14,17,117,118,119,199,346,347,348] + list(range(468,543))

### Función para Cargar y Estructurar Datos desde Parquet

In [ ]:
def Load_data(pq_path):
    # Columnas necesarias para reconstruir los landmarks
    data_columns = ['x', 'y', 'z', 'type', 'landmark_index', 'frame']
    df = pd.read_parquet(pq_path, columns=data_columns)

    # Reemplazamos valores NaN con 0.0 en las columnas numéricas (x, y, z)
    df[COLUMNS] = df[COLUMNS].fillna(0.0)
    
    # Ordenamos por frame, tipo de landmark (pose, hand, etc.), y su índice
    df = df.sort_values(by=['frame', 'type', 'landmark_index']).reset_index(drop=True)

    # Extraemos solo las columnas numéricas y convertimos a NumPy
    data = df[COLUMNS].values.astype(np.float32)

    # Calculamos cuántos landmarks hay por frame (asumimos que todos los frames tienen el mismo número)
    n_landmarks_per_frame = df['frame'].value_counts().iloc[0]
    
    # Número total de frames en el archivo
    n_frames = len(df) // n_landmarks_per_frame
    
    # Reformateamos el array a 3D: (frames, landmarks por frame, dimensiones)
    data = data.reshape(n_frames, n_landmarks_per_frame, DIMS)
    
    return data


### Clase `dataPreprocess`: Preprocesamiento de Secuencias de Landmarks

Esta clase encapsula el preprocesamiento necesario para las secuencias de video del dataset. Incluye:

- Interpolación de frames vacíos.
- Selección de landmarks relevantes.
- Padding para normalizar la longitud de la secuencia.
- Upsampling con media para ajustar secuencias largas a un tamaño fijo.

Se usa para transformar cualquier video en una entrada válida de tamaño `(INPUT_SIZE, len(LANDMARK_IDX), 3)`.


In [ ]:
class dataPreprocess:
    def __init__(self, input_size=INPUT_SIZE, max_gap=GAP, landmark_idxs=LANDMARK_IDX):
        self.input_size = input_size  # Longitud objetivo de cada secuencia
        self.max_gap = max_gap        # Máximo número de frames consecutivos permitidos con datos perdidos para interpolar
        self.landmark_idxs = landmark_idxs  # Índices de landmarks seleccionados

    def interpolate_missing(self, seq):
        seq = seq.copy()
        mask = np.all(seq == 0.0, axis=(1, 2))  # Frames vacíos = todos los valores en (x, y, z) son 0
        i = 0
        while i < len(seq):
            if mask[i]:  # Si es un frame vacío
                start = i
                while i < len(seq) and mask[i]:  # Buscar final del tramo vacío
                    i += 1
                end = i
                gap = end - start
                # Si el gap es muy grande o está al inicio/final, no se interpola
                if start == 0 or end == len(seq) or gap > self.max_gap:
                    continue
                # Interpolación lineal entre los dos extremos
                for j in range(gap):
                    alpha = (j + 1) / (gap + 1)
                    seq[start + j] = (1 - alpha) * seq[start - 1] + alpha * seq[end]
            else:
                i += 1
        return seq

    def pad(self, video, pad_left, pad_right):
        # Añade padding replicando el primer y último frame
        return np.concatenate([
            np.repeat(video[:1], pad_left, axis=0),
            video,
            np.repeat(video[-1:], pad_right, axis=0)
        ], axis=0)

    def __call__(self, video):
        # Paso 1: Interpolar frames perdidos
        video = self.interpolate_missing(video)

        # Paso 2: Filtrar solo los landmarks seleccionados
        if self.landmark_idxs is not None:
            video = video[:, self.landmark_idxs, :]

        T, L, D = video.shape  # T: frames, L: landmarks, D: dimensiones
        N = self.input_size    # Longitud objetivo

        if T < N:
            # Si hay menos frames que N, hacer padding
            pad_total = N - T
            pad_left = pad_total // 2
            pad_right = pad_total - pad_left
            video = self.pad(video, pad_left, pad_right)
            return video.astype(np.float32)

        if T == N:
            # Ya tiene la longitud deseada
            return video.astype(np.float32)

        # Si hay demasiados frames, hacemos upsampling para luego reducir con media
        repeat_factor = (N * N) // T
        video = np.repeat(video, repeats=repeat_factor, axis=0)

        # Recortar y hacer padding si es necesario
        T = video.shape[0]
        excess = T % N
        if excess > 0:
            pad_total = N - excess
            pad_left = pad_total // 2
            pad_right = pad_total - pad_left
            video = self.pad(video, pad_left, pad_right)

        # Promediar en bloques para reducir a longitud N
        video = video.reshape(N, -1, L, D)
        return video.mean(axis=1).astype(np.float32)

# Instanciamos la clase de preprocesamiento
preprocess_layer = dataPreprocess()

In [ ]:
# Función `get_data`: Cargar y Preprocesar un Archivo Individual

def get_data(file_path):
    # Cargar los datos crudos desde archivo Parquet (landmarks por frame)
    data = Load_data(file_path)
    
    # Aplicar preprocesamiento: interpolación, padding y selección de landmarks
    data = preprocess_layer(data)
    
    return data


##  Función `preprocess_data`: Construcción del Dataset Preprocesado

Esta función recorre todas las muestras del dataset, carga y transforma cada video usando `get_data`, y guarda los arrays procesados (`X`, `y`) en disco. Además, realiza una división en conjunto de entrenamiento y validación utilizando `GroupShuffleSplit` para asegurar que los participantes no se repitan entre ambos conjuntos.

In [ ]:
def preprocess_data(train, get_data, input_size=64, n_cols=None, n_dims=3, save_dir="."):
    N_SAMPLES = len(train)

    # Si no se especifica el número de columnas, se infiere con la primera muestra
    if n_cols is None:
        sample = get_data(train['file_path'].iloc[0])
        n_cols = sample.shape[1]

    # Inicializamos los arrays X e y
    X = np.zeros((N_SAMPLES, input_size, n_cols, n_dims), dtype=np.float32)  # Datos de entrada
    y = np.zeros(N_SAMPLES, dtype=np.int32)                                  # Etiquetas (sign_ord)

    # Procesamos cada archivo individualmente
    for row_idx, (file_path, sign_ord) in enumerate(tqdm(train[['file_path', 'sign_ord']].values)):
        if row_idx % 5000 == 0:
            print(f'Procesados: {row_idx}/{N_SAMPLES}')

        try:
            data = get_data(file_path)

            # Saltar muestras que contienen valores NaN
            if np.isnan(data).any():
                print(f"[NaN detectado] en fila {row_idx}: {file_path}")
                continue

            X[row_idx] = data
            y[row_idx] = sign_ord

        except Exception as e:
            print(f"[Error en {file_path}]: {e}")
            continue

    # Guardar arrays completos
    np.save(f"{save_dir}/X.npy", X)
    np.save(f"{save_dir}/y.npy", y)

    # División en train/val asegurando que participantes no se repitan
    splitter = GroupShuffleSplit(test_size=0.1, n_splits=1, random_state=SEED)
    participant_ids = train['participant_id'].values
    train_idxs, val_idxs = next(splitter.split(X, y, groups=participant_ids))

    # Guardar conjuntos train y val
    np.save(f"{save_dir}/X_train.npy", X[train_idxs])
    np.save(f"{save_dir}/y_train.npy", y[train_idxs])
    np.save(f"{save_dir}/X_val.npy",   X[val_idxs])
    np.save(f"{save_dir}/y_val.npy",   y[val_idxs])

    # Mostrar información básica sobre los conjuntos generados
    print(f"Participant ID Intersección train/val: {set(participant_ids[train_idxs]).intersection(participant_ids[val_idxs])}")
    print(f"X_train shape: {X[train_idxs].shape}, X_val shape: {X[val_idxs].shape}")
    print(f"y_train shape: {y[train_idxs].shape}, y_val shape: {y[val_idxs].shape}")


In [ ]:
# Ejecutamos todo el pipeline de preprocesamiento
preprocess_data(train, get_data)

## 💾 Exportación del Dataset Preprocesado

En esta última parte del notebook, se ha realizado la exportación completa del dataset de landmarks procesados para el proyecto **Sign2Speech**:

1. **Organización de archivos**: Los arrays generados (`X`, `y`, `X_train`, `y_train`, `X_val`, `y_val`) se han movido a una carpeta llamada `tfg_asl_preprocessed`.

2. **Generación de metadata**: Se creó el archivo `dataset-metadata.json` con los campos requeridos por la API de Kaggle, incluyendo el título, licencia y visibilidad privada del dataset.

3. **Configuración de la API de Kaggle**: Se configuró la autenticación mediante el archivo `kaggle.json` copiado desde los inputs del entorno.

4. **Subida del dataset a Kaggle**: Finalmente, se utilizó el comando `kaggle datasets create` para subir la carpeta `tfg_asl_preprocessed` a tu cuenta de Kaggle como un dataset privado.

Este proceso asegura que el dataset pueda ser accedido de forma remota desde cualquier entorno de entrenamiento o despliegue, manteniendo la trazabilidad y reutilización de los datos de forma segura y estructurada.

In [ ]:
# Crear carpeta si no existe
os.makedirs("tfg_asl_preprocessed", exist_ok=True)

# Mover archivos .npy a la carpeta
!mv X.npy tfg_asl_preprocessed/
!mv y.npy tfg_asl_preprocessed/
!mv X_train.npy tfg_asl_preprocessed/
!mv y_train.npy tfg_asl_preprocessed/
!mv X_val.npy tfg_asl_preprocessed/
!mv y_val.npy tfg_asl_preprocessed/

In [ ]:
metadata = {
    "title": "ASL Processed Landmarks TFG (Private)",
    "id": "aymaneelamranidl/asl-processed-landmarks",
    "licenses": [{"name": "CC0-1.0"}],
    "isPrivate": True
}

import json
with open("tfg_asl_preprocessed/dataset-metadata.json", "w") as f:
    json.dump(metadata, f)

In [ ]:
# Crear carpeta de configuración de Kaggle si no existe
!mkdir -p ~/.kaggle

# Copiar archivo de autenticación desde input
!cp /kaggle/input/keyfile/kaggle.json ~/.kaggle/

# Ajustar permisos (obligatorio)
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Crear el dataset en Kaggle desde la carpeta con los .npy y el metadata
!kaggle datasets create -p tfg_asl_preprocessed